In [5]:
import base64
import hmac
import json
import time
import requests
import toml

from urllib.parse import urljoin

# params
base_url = 'https://www.dojqutmbiuuo.com'
api_config = toml.load('../config/api.toml')
api_key = "9273c114-2d93-4167-95d1-c78093bd8065"
secret_key = "688CFC37EFA8CDE1A00360918BCD5195"
passphrase = "A5f6e778dx#"

In [6]:
def request(method, uri, params=None, body=None, headers=None, auth=False):
    """Initiate network request
   @param method: request method, GET / POST / DELETE / PUT
   @param uri: request uri
   @param params: dict, request query params
   @param body: dict, request body
   @param headers: request http header
   @param auth: boolean, add permission verification or not
   """
    if params:
        query = "&".join(
            ["{}={}".format(k, params[k]) for k in sorted(params.keys())]
        )
        uri += "?" + query
    url = urljoin(base_url, uri)

    if auth:
        timestamp = (
                str(time.time()).split(".")[0]
                + "."
                + str(time.time()).split(".")[1][:3]
        )
        if body:
            body = json.dumps(body)
        else:
            body = ""
        message = str(timestamp) + str.upper(method) + uri + str(body)
        mac = hmac.new(
            bytes(secret_key, encoding="utf8"),
            bytes(message, encoding="utf-8"),
            digestmod="sha256",
        )
        d = mac.digest()
        sign = base64.b64encode(d)

        if not headers:
            headers = {}
        headers["Content-Type"] = "application/json"
        headers["OK-ACCESS-KEY"] = api_key
        headers["OK-ACCESS-SIGN"] = sign
        headers["OK-ACCESS-TIMESTAMP"] = str(timestamp)
        headers["OK-ACCESS-PASSPHRASE"] = passphrase
    result = requests.request(
        method, url, data=body, headers=headers, timeout=10
    ).json()
    if result.get("code") and result.get("code") != "0":
        return None, result
    return result, None

In [7]:
def get_asset(currency):
    params = {"ccy": currency}
    result = request(
        "GET", "/api/v5/account/balance", params=params, auth=True
    )
    return result


get_asset('USDT')

({'code': '0',
  'data': [{'adjEq': '',
    'borrowFroz': '',
    'details': [{'accAvgPx': '',
      'availBal': '139.51055233416153',
      'availEq': '139.51055233416153',
      'borrowFroz': '',
      'cashBal': '139.51055233416153',
      'ccy': 'USDT',
      'clSpotInUseAmt': '',
      'crossLiab': '',
      'disEq': '139.49799638445145',
      'eq': '139.51055233416153',
      'eqUsd': '139.49799638445145',
      'fixedBal': '0',
      'frozenBal': '0',
      'imr': '0',
      'interest': '',
      'isoEq': '0',
      'isoLiab': '',
      'isoUpl': '0',
      'liab': '',
      'maxLoan': '',
      'maxSpotInUse': '',
      'mgnRatio': '',
      'mmr': '0',
      'notionalLever': '0',
      'openAvgPx': '',
      'ordFrozen': '0',
      'rewardBal': '0',
      'smtSyncEq': '0',
      'spotBal': '',
      'spotCopyTradingEq': '0',
      'spotInUseAmt': '',
      'spotIsoBal': '0',
      'spotUpl': '',
      'spotUplRatio': '',
      'stgyEq': '0',
      'totalPnl': '',
      'total

In [5]:
def get_grid_order_list(order_type):
    params = {"algoOrdType": order_type}
    result = request(
        "GET", "/api/v5/tradingBot/grid/orders-algo-pending", params=params, auth=True
    )
    return result


# get spot grid
result = get_grid_order_list(order_type='grid')
result

({'code': '0', 'data': [], 'msg': ''}, None)

In [6]:
# get future grid
result = get_grid_order_list(order_type='contract_grid')
result[0]['data']

[]

In [7]:
def get_grid_order_details(order_type, algo_id):
    params = {
        "algoOrdType": order_type,
        "algoId": algo_id,
    }
    result = request("GET", "/api/v5/tradingBot/grid/orders-algo-details", params=params, auth=True)
    return result


result = get_grid_order_details('grid', 687405759617044480)
result

(None,
 {'code': '51291',
  'data': [],
  'msg': 'The bot doesn’t exist or has already stopped'})

In [8]:
def get_recurring_orders():
    result = request("GET", "/api/v5/tradingBot/recurring/orders-algo-pending", auth=True)
    return result

result = get_recurring_orders()
result

({'code': '0', 'data': [], 'msg': ''}, None)

In [9]:
def get_grid_sub_orders(order_type, algo_id, order_status):
    params = {
        "algoOrdType": order_type,
        "algoId": algo_id,
        "type": order_status,
    }

    result = request("GET", "/api/v5/tradingBot/grid/sub-orders", params=params, auth=True)
    return result

result = get_grid_sub_orders('contract_grid', '688176870327844864', "live")
len(result[0]['data'])
result[0]['data'][0]

TypeError: 'NoneType' object is not subscriptable

In [10]:
def get_grid_positions(order_type, algo_id):
    params = {
        "algoOrdType": order_type,
        "algoId": algo_id,
    }

    result = request("GET", "/api/v5/tradingBot/grid/positions", params=params, auth=True)
    return result

result = get_grid_positions('contract_grid', '687008144513568768')
result

(None,
 {'code': '51291',
  'data': [],
  'msg': 'The bot doesn’t exist or has already stopped'})

# SPOT DAILY KLINE DATA

In [20]:
def get_spot_klines(spot, period, limit):
    params = {
        "instId": spot,
        "bar": period,
        "limit": limit,
    }
    
    result = request("GET", "/api/v5/market/history-index-candles", params=params, auth=True)
    return result
    # 开，高，低，收

result = get_spot_klines('BTC-USDT', '1D', 100)
result[0]['data']

[['1724083200000', '58539', '61395.7', '58378.4', '58827.6', '1'],
 ['1723996800000', '59863.4', '59946.7', '57811.5', '58539', '1'],
 ['1723910400000', '59390', '60277.3', '59153.3', '59862.2', '1'],
 ['1723824000000', '58073.6', '59819.8', '57996', '59390', '1'],
 ['1723737600000', '59686.2', '59849.7', '56114.8', '58074.1', '1'],
 ['1723651200000', '59433.5', '59774.3', '57715.8', '59686.9', '1'],
 ['1723564800000', '59477.9', '61770.7', '58827.2', '59431.1', '1'],
 ['1723478400000', '60206.6', '60413.9', '58417.5', '59477.9', '1'],
 ['1723392000000', '60126.8', '60697', '57641.4', '60206.6', '1'],
 ['1723305600000', '60517.9', '61850.8', '59939.7', '60126.8', '1'],
 ['1723219200000', '59810.4', '61472.5', '59728.5', '60517.9', '1'],
 ['1723132800000', '59428.1', '62739.6', '59241', '59810.3', '1'],
 ['1723046400000', '56098.4', '59508.8', '54558.5', '59427.6', '1'],
 ['1722960000000', '56406.9', '57730.1', '55565.7', '56097.9', '1'],
 ['1722873600000', '54317.4', '56610.9', '52551.

# Save Different Spots Daily Data

In [43]:
import pandas as pd

spot_pool = ['BTC-USDT', 'ETH-USDT', 'SOL-USDT', 'BNB-USDT']
for spot in spot_pool:
    data_dict = {
        'timestamp': [],
        'open': [],
        'high': [],
        'low': [],
        'close': [],
    }
    result = get_spot_klines(spot, '1D', 1000)
    result = result[0]['data']
    for data in reversed(result):
        keys = list(data_dict.keys())
        for i in range(5):
            data_dict[keys[i]].append(data[i]) 
    df = pd.DataFrame(data_dict)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.to_csv(f'assets/{spot}.csv', index=False)

C:\Users\62552\AppData\Local\Temp\ipykernel_4980\3295598858.py:19: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
C:\Users\62552\AppData\Local\Temp\ipykernel_4980\3295598858.py:19: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
C:\Users\62552\AppData\Local\Temp\ipykernel_4980\3295598858.py:19: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing s

# UNIX Timestamp => Normal Timestamp

In [33]:
import time
time_unix = 1715529600000
time_normal = time.gmtime(time_unix)
dt = time.strftime("%Y-%m-%d", time_normal)
dt

OSError: [Errno 22] Invalid argument